In [1]:
import cv2
import tensorflow.keras as keras
import numpy as np
from PIL import Image
import os
import numpy as np
import cv2
import os
import h5py
import dlib
from imutils import face_utils
from keras.models import load_model
import sys
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.utils import to_categorical
from keras import backend as K 
from sklearn.model_selection import train_test_split
from keras import callbacks
import tensorflow as tf
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

Using TensorFlow backend.


In [ ]:
image_size = 224
IMG_SHAPE = (image_size, image_size, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights='imagenet')
base_model.trainable = False

In [ ]:
classes = 2 #Change this as per the number of known persons we want to store in our database 
model = tf.keras.Sequential([
                          base_model,
                          keras.layers.GlobalAveragePooling2D(),
                          keras.layers.Dense(classes, activation='softmax')])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
model.summary()

## Testing different face detector models

In [ ]:
import cv2
import dlib
from mtcnn.mtcnn import MTCNN
import numpy as np

detector1 = MTCNN()
detector2 = dlib.get_frontal_face_detector()
modelFile = "D:/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "D:/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
classifier2 = cv2.CascadeClassifier('D:/haarcascade_frontalface2.xml')

cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX 
while(True):
    ret, img = cap.read()
    if ret == True:
        img = cv2.resize(img, None, fx=0.5, fy=0.5)
        height, width = img.shape[:2]
        img1 = img.copy()
        img2 = img.copy()
        img3 = img.copy()
        # detect faces in the image
        faces1 = detector1.detect_faces(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces2 = detector2(gray, 1)
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),
                                        1.0, (300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces3 = net.forward()
        faces4 = classifier2.detectMultiScale(img)
        
        # display faces on the original image
        for result in faces1:
            x, y, w, h = result['box']
            x1, y1 = x + w, y + h
            cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)
        cv2.putText(img, 'mtcnn', (30, 30), font, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
        for result in faces2:
            x = result.left()
            y = result.top()
            x1 = result.right()
            y1 = result.bottom()
            cv2.rectangle(img1, (x, y), (x1, y1), (0, 0, 255), 2)
        cv2.putText(img1, 'dlib', (30, 30), font, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
        for i in range(faces3.shape[2]):
            confidence = faces3[0, 0, i, 2]
            if confidence > 0.5:
                box = faces3[0, 0, i, 3:7] * np.array([width, height, width, height])
                (x, y, x1, y1) = box.astype("int")
                cv2.rectangle(img2, (x, y), (x1, y1), (0, 0, 255), 2)
        cv2.putText(img2, 'dnn', (30, 30), font, 1, (255, 255, 0), 2, cv2.LINE_AA)
                
        for result in faces4:
            x, y, w, h = result
            x1, y1 = x + w, y + h
            cv2.rectangle(img3, (x, y), (x1, y1), (0, 0, 255), 2)
        cv2.putText(img3, 'haar', (30, 30), font, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
        h1 = cv2.hconcat([img3, img1])
        h2 = cv2.hconcat([img, img2])
        fin = cv2.vconcat([h1, h2])

        cv2.imshow("mtcnn", img)
        cv2.imshow("dlib", img1)
        cv2.imshow("dnn", img2)
        cv2.imshow("haar", img3)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break


           
cap.release()
cv2.destroyAllWindows()

## Preparing Training Dataset

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('D:/haarcascade_frontalface2.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (300, 300))

        # Save file in specified directory with unique name
        file_name_path = r"D:/dataset_for_face/Asha_" + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 1000: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Samples Taken")

## Training our model

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from keras import callbacks
from keras.models import Sequential, model_from_yaml, load_model
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D
from keras.optimizers import Adam, SGD
from keras.utils import np_utils, plot_model
from keras.applications.resnet50 import preprocess_input, decode_predictions
import cv2


np.random.seed(44) #MALATYA<3 
img_h, img_w = 224, 224
image_size = (128, 128)
nbatch_size = 2
nepochs = 100
nb_classes = 2
def load_data():
    path = 'D:/dataset_for_face/'
    files = os.listdir(path)
    images = []
    labels = []
    for f in files:
        img_array = cv2.imread(os.path.join(path,f))
        new_array = cv2.resize(img_array, (img_h, img_w))
        images.append(new_array)

        if 'Asha' in f:
            labels.append(0)
        elif 'Chiku' in f:
            labels.append(1)
       
    data = np.array(images)
    data = np.array(data).reshape(-1, img_h, img_h, 3)
    labels = np.array(labels)

    labels = np_utils.to_categorical(labels, 2)
    return data, labels
print("load_data......")
x_train, y_train = load_data()
x_train = x_train/255
print(x_train.shape,y_train.shape)
checkpoint = callbacks.ModelCheckpoint("abcd.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = callbacks.EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]
model.fit(x_train, y_train, batch_size=nbatch_size, callbacks=callbacks, epochs=nepochs, verbose=1, validation_split=0.1)

# Evaluating the model
print("evaluate......")
score, accuracy = model.evaluate(x_train[:3], y_train[:3], batch_size=nbatch_size)
print('score:', score, 'accuracy:', accuracy)

# Saving the mofel and making the reaining process much easier and mre practial. Saving the weights separately： from
# keras.models import model_from_yaml, load_model


## Loading Model

In [1]:
from tensorflow import keras
model = keras.models.load_model("abcd.h5")

## Making Predictions 

In [2]:
import cv2
import numpy as np
import os
from datetime import datetime
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from keras.models import Sequential, model_from_yaml, load_model
from keras.optimizers import Adam, SGD
import cv2

detector = cv2.CascadeClassifier('D:/haarcascade_frontalface2.xml')
img_h, img_w = 224,224


capture = cv2.VideoCapture(0)

while True:
    ret, frame = capture.read()
    faces = detector.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        ROI = frame[y:y + h, x:x + w]
        for f in faces:
            new_array = cv2.resize(ROI, (img_h, img_w))
            data = np.array(new_array)
            data = np.array(data).reshape(-1, img_h, img_h, 3)
            result = model.predict_classes(data, verbose=0)

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            if result[0] == 0:
                cv2.putText(frame, 'Chiku', (x, y), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 2)
            if result[0] == 1:
                cv2.putText(frame, 'Asha', (x, y), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 2)
            
                
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

capture.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

# from PIL import ImageGrab

path = 'Training_images'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)


def findEncodings(images):
    encodeList = []


    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
    

def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

#### FOR CAPTURING SCREEN RATHER THAN WEBCAM
# def captureScreen(bbox=(300,300,690+300,530+300)):
#     capScr = np.array(ImageGrab.grab(bbox))
#     capScr = cv2.cvtColor(capScr, cv2.COLOR_RGB2BGR)
#     return capScr

encodeListKnown = findEncodings(images)
print('Encoding Complete')

cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
# img = captureScreen()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
# print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
# print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)

    cv2.imshow('Webcam', img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

['Asha.jpg']
['Asha']
Encoding Complete


In [ ]:
 if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
        if not matches[matchIndex]:
            name = 'Un'
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

In [11]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb  6 21:25:27 2021

@author: hp
"""
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import time
fpsLimit = 1 # throttle limit

path = 'Training_images1'
#path = 'Training_images2'
images = []
personNames = []
myList = os.listdir(path)
print(myList)
for cu_img in myList:
    current_Img = cv2.imread(f'{path}/{cu_img}')
    images.append(current_Img)
    personNames.append(os.path.splitext(cu_img)[0])
print(personNames)


def faceEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


def attendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                time_now = datetime.now()
                tStr = time_now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{tStr}')


encodeListKnown = faceEncodings(images)
print('All Encodings Complete!!!')

cap = cv2.VideoCapture(0)
startTime = time.time()

while True:
    ret, frame = cap.read()
    nowTime = time.time()
    if (int(nowTime - startTime)) > fpsLimit:
        faces = cv2.resize(frame, (0, 0), None, 0.25, 0.25)
        faces = cv2.cvtColor(faces, cv2.COLOR_BGR2RGB)

        facesCurrentFrame = face_recognition.face_locations(faces)
        encodesCurrentFrame = face_recognition.face_encodings(faces, facesCurrentFrame)

        for encodeFace, faceLoc in zip(encodesCurrentFrame, facesCurrentFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            # print(faceDis)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = personNames[matchIndex].upper()
                # print(name)
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            if not matches[matchIndex]:
                name = 'Unknown'
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                t=time.strftime("%Y-%m-%d_%H-%M-%S")
                file=t+'.jpg'
                cv2.imwrite(file,frame)
                attendance(name)
            startTime = time.time()

            cv2.imshow('Webcam', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

['obama.jpg']
['obama']
All Encodings Complete!!!


In [3]:
import time
import cv2
from datetime import datetime
fpsLimit = 1 # throttle limit
startTime = time.time()
cv = cv2.VideoCapture(0)
while True:
    ret, frame = cv.read()
    nowTime = time.time()
    if (int(nowTime - startTime)) > fpsLimit:
        t=time.strftime("%Y-%m-%d_%H-%M-%S")
        file=t+'.jpg'
        cv2.imwrite(file,frame)
        startTime = time.time() # reset time
        
    cv2.imshow('Webcam', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cv.release()
cv2.destroyAllWindows()